# Simulating child allowances in US states

Poverty has a lasting impact on children.  Growing up in poverty increases [stress](https://heckmanequation.org/resource/invest-in-early-childhood-development-reduce-deficits-strengthen-the-economy/) and [incarceration rates](https://heckmanequation.org/resource/invest-in-early-childhood-development-reduce-deficits-strengthen-the-economy/) and decreases [educational](https://www.sciencedirect.com/science/article/abs/pii/S1876285915003836) and [health](https://www.aeaweb.org/articles?id=10.1257/app.2.1.86) outcomes.  These negative consequences not only impact the nation’s poor, but the entire economy as well--child poverty shrinks GDP by more than [\$1 trillion annually](https://www.nap.edu/read/25246/chapter/1).

Research has shown that giving money to families with children, as most developed countries do, can reduce each of these issues.  Programs that provide cash to families with children are called a _child allowance_, and are typically paid out on a monthly basis.  

This site allows users to explore the impact of a potential child allowance of various amounts in each state.  Users can also select between three funding mechanisms:

* **Deficit funded** does not impose any new taxes.  Note that each of these tax reforms is static and does not account for changes in behavior.
* **Federal tax** as a flat rate on taxable income. For each child allowance amount the total cost is calculated by multiplying the annual child allowance by the total number of children.  To calculate the revenue neutral tax rate, the total cost is divided by the nation’s total taxable income.  For example, a child allowance of \$100 per month would cost about \$88 billion annually and require a new flat tax of 1.1 percent.
* **State tax** as a flat rate on taxable income. The state tax is calculated in the same manner as the federal tax, but at the state level. This shows how states can fund their own child allowances. Because states vary in child population and income, different states have different tax rates. A \$100 monthly child allowance in DC is offset by a 0.5 percent tax on taxable income while the same amount requires a 1.3 percent tax in Alabama.

In [9]:
# TODO:
# Separate tax reform and Gini charts
# Change labels
# Add poverty maps


import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

summary = pd.read_csv('https://github.com/ngpsu22/Child_Allowance_States/raw/main/poverty_gini_tax_child_allowance')
summary.drop('Unnamed: 0', 1, inplace = True)

tax = summary[(summary.race == 'All') & (summary.age_group == 'all')]

x = tax.child_allowance.unique()
state_names = tax.state.unique()
default_state = state_names[0]

REFORM = {'state_tax_rate': 'State tax rate', 
          'fed_tax_rate': 'Federal tax rate'}
GINI = {'fed_gini': 'Gini index - federal tax',
        'state_gini': 'Gini index - state tax',
        'non_funded_gini': 'Gini index - deficit funded'}

data_list = []
for state in state_names:
    state_data = tax[tax.state == state]
    state_list = []
    for reform in REFORM:
        state_list.append(state_data[reform])
    for gini in GINI:
        state_list.append(state_data[gini])
    data_list.append(state_list)

data_columns = list(REFORM.keys()) + list(GINI.keys())
data = pd.DataFrame(data_list, columns = data_columns)
data['State'] = state_names
data = data.set_index('State')

def getDataList(state):
    data_list = []
    for dc in data_columns:
        data_list.append(data.loc[state][dc])
    return data_list

fig = make_subplots(rows=2, cols=1, 
                    subplot_titles=(
                        'Revenue neutral tax rates in ' + default_state,
                        'Gini index in' + default_state))

for reform in REFORM:
    fig.add_trace(go.Scatter(
        x=x, 
        y=tax[tax.state == default_state][reform],
        name=REFORM[reform]),
        row = 1, col = 1
    )
for gini in GINI:
    fig.add_trace(go.Scatter(
        x=x, 
        y=tax[tax.state == default_state][gini],
        name=GINI[gini]),
        row = 2, col = 1
    )
    
buttons = []
for state in state_names:
    new_button = {'method': 'update',
                  'label': state,
                  'args': [{'y': getDataList(state)},
                           {'annotations[0].text':
                            'Revenue neutral tax rates in ' + state,
                            'annotations[1].text':
                            'Gini index in ' + state}
                          ]}
    buttons.append(new_button)
    
# construct menus
updatemenus = [{'buttons': buttons,
                'direction': 'down',
                'showactive': True,
                'pad':{"r": 50},
               }]

# update layout with buttons, and show the figure
fig.update_layout(updatemenus=updatemenus)

fig.update_xaxes(title_text='Monthly child allowance', row=1, col=1)
fig.update_xaxes(title_text='Monthly child allowance', row=2, col=1)
fig.update_yaxes(title_text='Tax rate on taxable income', row=1, col=1)
fig.update_yaxes(title_text='Gini index', row=2, col=1)

fig.update_layout(yaxis_ticksuffix='%',
                  font=dict(family='Roboto'),
                  hovermode='x', 
                  xaxis_tickprefix='$',
                  xaxis_ticksuffix='',
                  plot_bgcolor='white',
                  legend_title_text='')

fig.update_traces(mode='markers+lines', hovertemplate=None)

fig.update_layout(height=600, margin=dict(l=0, r=0, t=100, b=0),
                  title_text="Tax reform and Gini index")
fig

While the new tax would hit most Americans, save the very poorest who earn less than the standard deduction, the overall distributional consequences would be positive.  For instance, the bottom decile would see their average resources rise by ________ per person with a federal $300 per month child allowance while the top 10 percent would see their average resources per person fall by 

In [2]:
deciles = pd.read_csv('data/deciles.csv')

# Make chart symmetric with boundary at the maximum.
boundary = deciles.net_chg.agg([min, max]).abs().max()

# TODO: Make this work for all selections with a drop-down.

test = deciles.query('state == "US" & funding == "federal"').round()

fig = px.bar(test, x='spm_resources_pp_decile',
             y='net_chg',
             animation_frame='monthly_ca',
             labels={'monthly_ca': 'Monthly child allowance'})
fig.update_layout(
    title='Average net change per dollar child allowance by decile',
    xaxis_title='Decile of resources per person',
    yaxis_title='Average annual net change per dollar child allowance',
    yaxis_tickprefix='$',
    plot_bgcolor='white',
    font=dict(family='Roboto')
)
fig.update_yaxes(range=[-boundary, boundary])
fig.update_traces(hovertemplate=None)
fig.show(config={'displayModeBar': False})

In [ ]:
# Make chart symmetric with boundary at the maximum.
pct_boundary = deciles.pct_chg.agg([min, max]).abs().max()

# TODO: Make this work for all selections with a drop-down.

test = deciles.query('state == "US" & funding == "federal"').round()

fig = px.bar(test, x='spm_resources_pp_decile',
             y='net_chg',
             animation_frame='monthly_ca',
             labels={'monthly_ca': 'Monthly child allowance'})
fig.update_layout(
    title='Average net change per dollar child allowance by decile',
    xaxis_title='Decile of resources per person',
    yaxis_title='Average annual net change per dollar child allowance',
    yaxis_tickprefix='$',
    plot_bgcolor='white',
    font=dict(family='Roboto')
)
fig.update_yaxes(range=[-boundary, boundary])
fig.update_traces(hovertemplate=None)
fig.show(config={'displayModeBar': False})

The Gini coefficient is another tool to calculate a policy reform’s impact on inequality.  It lies between 0 and 1, with 0 indicating that everyone has the exact same income, and 1 indicating that one person possesses all the income.  

Across funding strategies, child allowances reduce inequality as measured by the Gini coefficient, with larger child allowances producing larger inequality reductions. Nationally, a \$500 monthly child allowance shrinks the Gini coefficient nine percent, from 0.446 to 0.406.  At the state level, current Gini coefficients vary, but a \$200 monthly child allowance tends to cut inequality between 4 and 5 percent.

In [3]:
# TODO: Insert Gini chart, separated from the prior.

A child allowance can also substantially cut poverty.  In this example, a person is said to be in poverty if their household’s total post tax and transfer income is less than their poverty threshold.  Poverty thresholds are determined by the Census Bureau’s Supplemental Poverty Measure (SPM), which considers a medley of factors including household size, housing status, and local cost of living.

In [4]:
# TODO: Combine these with a drop-down.

no_fund_states = summary[summary['age_group'] == 'child']
no_fund_states = summary[summary['race'] == 'All']

no_fund_map = px.choropleth(no_fund_states, 
                       locations='code',
                       color='non_funded_poverty_rate', 
                       animation_frame='child_allowance',
                       color_continuous_scale='reds',
                       locationmode='USA-states',
                       scope='usa',
                       range_color=(0, 22),
                       title=('Child poverty with with no tax reform'),
                       height=600,
                       labels={'child_allowance':'Child Allowance',
                              'code': 'State',
                              'non_funded_poverty_rate': 'Child poverty rate'})

no_fund_map.update_layout(coloraxis_showscale=False,
                          font=dict(family='Roboto'))
no_fund_map.show(config={'displayModeBar': False})

In [5]:
fed_states = summary[summary['age_group'] == 'child']
fed_states = summary[summary['race'] == 'All']

fed_map = px.choropleth(fed_states, 
                        locations='code',
                        color='fed_poverty_rate', 
                        animation_frame='child_allowance',
                        color_continuous_scale='reds',
                        locationmode='USA-states',
                        scope='usa',
                        range_color=(0, 22),
                        title='Child poverty with federally funded tax reform',
                        height=600,
                        labels={'child_allowance':'Child allowance',
                                'code': 'State',
                                'fed_poverty_rate': 'Child poverty rate'})

fed_map.update_layout(coloraxis_showscale=False,
                      font=dict(family='Roboto'))
fed_map.show(config={'displayModeBar': False})

In [6]:
# create a choropleth map for state funded child allowance

state_states = summary[summary['age_group'] == 'child']
state_states = summary[summary['race'] == 'All']

state_map = px.choropleth(state_states, 
                       locations='code',
                       color='state_poverty_rate', 
                       animation_frame='child_allowance',
                       color_continuous_scale='reds',
                       locationmode='USA-states',
                       scope='usa',
                       range_color=(0, 22),
                       title=('Child poverty with state funded tax reform'),
                       height=600,
                       labels={'child_allowance':'Child Allowance',
                              'code': 'State',
                              'state_poverty_rate': 'Child poverty rate'})

state_map.update_layout(coloraxis_showscale=False,
                        font=dict(family='Roboto'))
state_map.show(config={'displayModeBar': False})

The next set of charts show how different demographics are impacted by each reform.  Across states, a $300 monthly child allowance cuts child poverty by about half and narrows the gap between Black and White child poverty within a few percentage points.

In [7]:
age_groups = summary[summary['race'] == 'All']
age_groups

,code,state,child_allowance,age_group,race,non_funded_poverty_rate,non_funded_tax_rate,non_funded_gini,fed_poverty_rate,fed_tax_rate,fed_gini,state_poverty_rate,state_tax_rate,state_gini
1,AL,Alabama,0,child,All,14.8,0.0,0.418,14.8,0.0,0.418,14.8,0.0,0.418
5,AL,Alabama,0,adult,All,12.1,0.0,0.440,12.1,0.0,0.440,12.1,0.0,0.440
9,AL,Alabama,0,all,All,12.7,0.0,0.446,12.7,0.0,0.446,12.7,0.0,0.446
13,AL,Alabama,50,child,All,13.7,0.0,0.409,13.7,0.5,0.408,13.7,0.7,0.408
17,AL,Alabama,50,adult,All,11.8,0.0,0.438,11.8,0.5,0.437,11.8,0.7,0.437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6845,US,US,450,adult,All,10.2,0.0,0.424,10.5,4.8,0.416,10.5,4.8,0.417
6849,US,US,450,all,All,9.0,0.0,0.418,9.2,4.8,0.410,9.2,4.8,0.411
6853,US,US,500,child,All,4.2,0.0,0.352,4.3,5.3,0.341,4.3,5.4,0.342
6857,US,US,500,adult,All,10.1,0.0,0.422,10.4,5.3,0.414,10.4,5.4,0.414


## Poverty - Race

In [8]:
race = summary[summary['age_group'] == 'child']

Together, these figures show how a child allowance can accomplish its initial purpose of reducing child poverty while also decreasing adult poverty and abating inequality.  

Currently, the closest thing America has to a child allowance is the Child Tax Credit which provides up to $2,000 per child to eligible families. Yet over one-third of children in the U.S are ineligible for the full child tax credit because their families don’t earn enough to qualify, and another one-tenth are excluded because their families earn too much.

The charts above reflect the opportunity for federal legislation to reduce child poverty across states, as well as states’ opportunity to reduce child poverty with truly universal child allowances of their own.  This data can be used to advocate for the expansion of child allowances and fully refundable tax credits to reduce poverty and prioritize the well-being of children.